This notebook is for downloading the resting-state, surface, thickness and volume data from the human connectome

In [ ]:
import pandas as pd
from os.path import join as opj
import subprocess
import os
import shutil

In [ ]:
local_subjects = pd.read_csv("../data/subjects_finally_used.txt", squeeze=True)

# Exctract release of each subject and save it somewhere
base_dir = "path/to/data/dir"

In [ ]:
def get_commands(sub_id, rel_name, output_dir):
    
    rel_resting_path = "MNINonLinear/Results/rfMRI_REST1_LR/rfMRI_REST1_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii"
    rel_surface_path = opj("T1w/fsaverage_LR32k/", sub_id + ".midthickness_MSMAll_va.32k_fs_LR.dscalar.nii")
    rel_thickness_path = opj("MNINonLinear/fsaverage_LR32k/", sub_id + ".thickness_MSMAll.32k_fs_LR.dscalar.nii")
    rel_volume_path = opj("T1w", sub_id, "stats/aseg.stats")
    
    
    _cmd_resting = ['aws', 's3', 'cp', 
                   opj('s3://hcp-openaccess', rel_name, sub_id, rel_resting_path),
                   output_dir]
    _cmd_surface = ['aws', 's3', 'cp', 
                   opj('s3://hcp-openaccess', rel_name, sub_id, rel_surface_path),
                   output_dir]
    _cmd_thickness = ['aws', 's3', 'cp', 
                   opj('s3://hcp-openaccess', rel_name, sub_id, rel_thickness_path),
                   output_dir]
    _cmd_volume = ['aws', 's3', 'cp', 
                   opj('s3://hcp-openaccess', rel_name, sub_id, rel_volume_path),
                   output_dir]
    
    return _cmd_resting, _cmd_surface, _cmd_thickness, _cmd_volume

In [ ]:
missing_subjects = []
for sub_id in local_subjects.astype(str):
    
    output_dir = opj(base_dir, "sub-" + sub_id)
    if os.path.exists(output_dir) is False:
        os.mkdir(output_dir)
    
    
    try:      
        cmd_resting, cmd_surface, cmd_thickness, cmd_volume = get_commands(sub_id, 'HCP_1200', output_dir)
        subprocess.run(cmd_resting)
        subprocess.run(cmd_surface)  
        subprocess.run(cmd_thickness)  
        subprocess.run(cmd_volume)
        
        if len(os.listdir(output_dir)) != 4:
            print("some file is missing")
            missing_subjects.append(sub_id)
            shutil.rmtree(output_dir)
            
    except:
        print("check subject " + sub_id)